In [ ]:
#compute the best asset allocation with fixed trigger, the flaw is annual relanace
# dont know how is it done

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Historical return & volatility for each NPS asset class (in %)
asset_classes = {
    "Equity (E)": {"mean": 12, "std": 18}, 
    "Corporate Bonds (C)": {"mean": 8, "std": 6},  
    "Govt Securities (G)": {"mean": 6, "std": 4},  
    "Alternative (A)": {"mean": 10, "std": 12}  
}

# Investment parameters
investment_years = 30  
simulations = 1000  
initial_investment = 10_00_000  # Initial corpus
monthly_contribution = 10_000  # Monthly NPS contribution
annual_growth_rate = 0.10  # 10% increase in contribution per year
months_per_year = 12  

# Define allocation constraints (NPS Active Choice Limits)
max_equity = 70  # Max 50% in Equity for Tier 1 NPS
max_bonds = 100
max_govt_sec = 100
max_alternatives = 5

# Different rebalancing triggers to test
rebalance_triggers = {
    "No Rebalancing": None,
    "Annual Rebalancing": 1,
    "Biannual Rebalancing": 2,
    "Drift > 5%": 0.05,
    "Drift > 10%": 0.10
}

# Seed for reproducibility
np.random.seed(42)

# Store results for different rebalancing strategies
strategy_results = {}

for strategy, rebalance_rule in rebalance_triggers.items():
    final_returns = []
    
    for _ in range(simulations):
        # Generate a random allocation within constraints
        eq = np.random.uniform(0, max_equity)
        cb = np.random.uniform(0, max_bonds)
        gs = np.random.uniform(0, max_govt_sec)
        al = np.random.uniform(0, max_alternatives)

        # Normalize to 100%
        total = eq + cb + gs + al
        eq, cb, gs, al = eq / total, cb / total, gs / total, al / total
        target_allocation = np.array([eq, cb, gs, al])  
        
        portfolio_value = initial_investment
        portfolio_allocations = target_allocation.copy()
        current_monthly_contribution = monthly_contribution
        
        for year in range(investment_years):
            for month in range(months_per_year):
                # Generate monthly returns for each asset
                monthly_returns = np.array([
                    np.random.normal(asset_classes["Equity (E)"]["mean"] / 12, asset_classes["Equity (E)"]["std"] / np.sqrt(12)),
                    np.random.normal(asset_classes["Corporate Bonds (C)"]["mean"] / 12, asset_classes["Corporate Bonds (C)"]["std"] / np.sqrt(12)),
                    np.random.normal(asset_classes["Govt Securities (G)"]["mean"] / 12, asset_classes["Govt Securities (G)"]["std"] / np.sqrt(12)),
                    np.random.normal(asset_classes["Alternative (A)"]["mean"] / 12, asset_classes["Alternative (A)"]["std"] / np.sqrt(12))
                ])

                # Apply returns to portfolio
                portfolio_value *= (1 + np.dot(portfolio_allocations, monthly_returns) / 100)

                # Add monthly contribution
                portfolio_value += current_monthly_contribution

                # Compute new allocation after growth
                asset_values = portfolio_value * portfolio_allocations
                new_allocations = asset_values / asset_values.sum()

                # Rebalancing logic
                if rebalance_rule is None:
                    pass  # No rebalancing
                elif isinstance(rebalance_rule, int) and year % rebalance_rule == 0 and month == 11:
                    portfolio_allocations = target_allocation.copy()  # Time-based rebalancing
                elif isinstance(rebalance_rule, float) and month == 11:
                    drift = new_allocations - target_allocation
                    if np.any(np.abs(drift) > rebalance_rule):  
                        portfolio_allocations = target_allocation.copy()  # Drift-based rebalancing

            # Increase monthly contribution by 10% annually
            current_monthly_contribution *= (1 + annual_growth_rate)

        final_returns.append(portfolio_value)

    strategy_results[strategy] = final_returns

# Convert to DataFrame for analysis
results_df = pd.DataFrame(strategy_results)
print(results_df)
# Find the best rebalancing strategy
best_strategy = results_df.mean().idxmax()
best_mean_return = results_df[best_strategy].mean()

print(f"\nBest Rebalancing Strategy: {best_strategy}")
print(f"Expected Portfolio Value: ₹{best_mean_return:,.2f}")

# Plot the distribution of final corpus values for different strategies
plt.figure(figsize=(12, 6))
for strategy in strategy_results.keys():
    plt.hist(results_df[strategy], bins=50, alpha=0.5, label=strategy)

plt.axvline(best_mean_return, color='red', linestyle='dashed', linewidth=2, label="Best Strategy")
plt.xlabel("Final Portfolio Value (₹)")
plt.ylabel("Frequency")
plt.title("Monte Carlo Simulation: Final Corpus Distribution Across Rebalancing Strategies")
plt.legend()
plt.show()


In [ ]:
#compute the impact of top-up on dips

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Simulation parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
market_dip_threshold = -0.05  # Dip defined as -2% monthly return
top_up_amount = 10000  # Extra ₹10k per dip

# Simulated monthly returns (Equity-heavy NPS)
mean_return = 0.008  # Avg monthly return ~9.6% annualized
volatility = 0.05  # Market volatility

# Running Monte Carlo Simulations
simulations = 10000
results_without_topup = []
results_with_topup = []

for _ in range(simulations):
    corpus_without_topup = 0
    corpus_with_topup = 0
    monthly_contribution_dynamic = monthly_contribution  # Contribution increases yearly

    for month in range(months):
        # Simulate market return
        market_return = np.random.normal(mean_return, volatility)

        # Without Top-Up: Regular contribution
        corpus_without_topup = corpus_without_topup * (1 + market_return) + monthly_contribution_dynamic

        # With Top-Up: Extra ₹10k investment during dips
        if market_return < market_dip_threshold:
            corpus_with_topup += top_up_amount  # Extra investment

        corpus_with_topup = corpus_with_topup * (1 + market_return) + monthly_contribution_dynamic

        # Increase monthly contribution annually
        if month % 12 == 0:
            monthly_contribution_dynamic *= annual_increase

    results_without_topup.append(corpus_without_topup)
    results_with_topup.append(corpus_with_topup)

# Convert to DataFrame
results_df = pd.DataFrame({
    "Without Top-Up": results_without_topup,
    "With Top-Up": results_with_topup
})

# Calculate mean corpus for both strategies
mean_without_topup = results_df["Without Top-Up"].mean()
mean_with_topup = results_df["With Top-Up"].mean()

print(f"Final Corpus Without Top-Up: ₹{mean_without_topup/1e7:.2f} Cr")
print(f"Final Corpus With Top-Up: ₹{mean_with_topup/1e7:.2f} Cr")

# Visualization
plt.figure(figsize=(12, 5))

# Histogram Without Top-Up
plt.subplot(1, 2, 1)
plt.hist(results_df["Without Top-Up"], bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.axvline(mean_without_topup, color='red', linestyle='dashed', linewidth=2)
plt.title("Final Corpus Without Top-Up")
plt.xlabel("Corpus (₹)")
plt.ylabel("Frequency")

# Histogram With Top-Up
plt.subplot(1, 2, 2)
plt.hist(results_df["With Top-Up"], bins=50, alpha=0.7, color='green', edgecolor='black')
plt.axvline(mean_with_topup, color='red', linestyle='dashed', linewidth=2)
plt.title("Final Corpus With Top-Up")
plt.xlabel("Corpus (₹)")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()


In [ ]:
#optimize allocation based on deviation

import random

def generate_returns(allocation, years=30, simulations=10000):
    """
    Simulates NPS returns for given asset allocation using Monte Carlo method.
    """
    asset_classes = {
        'Equity': (0.12, 0.20),  # Mean return 12%, volatility 20%
        'Corporate Bonds': (0.08, 0.10),
        'Government Bonds': (0.06, 0.05),
        'Alternative Assets': (0.10, 0.15)
    }
    
    final_values = []
    for _ in range(simulations):
        value = 0
        annual_contribution = 100000  # Starting annual contribution
        for year in range(1, years + 1):
            yearly_return = sum(
                allocation[asset] * (random.uniform(asset_classes[asset][0] - asset_classes[asset][1],
                                      asset_classes[asset][0] + asset_classes[asset][1]))
                for asset in allocation)
            value += annual_contribution
            value *= (1 + yearly_return)
            if year % 10 == 0:  # Increase contribution by 10% every 10 years
                annual_contribution *= 1.10
        final_values.append(value)
    return final_values

def optimize_allocation():
    """
    Finds the best asset allocation to maximize returns using Monte Carlo simulation.
    """
    best_allocation = None
    best_median_return = 0
    for _ in range(100):  # Try 100 different allocations
        allocation = {
            'Equity': random.uniform(0.2, 0.75),
            'Corporate Bonds': random.uniform(0.05, 0.4),
            'Government Bonds': random.uniform(0.05, 0.4),
            'Alternative Assets': random.uniform(0, 0.2)
        }
        total = sum(allocation.values())
        for key in allocation:
            allocation[key] /= total  # Normalize to ensure sum is 1
        returns = generate_returns(allocation)
        median_return = sorted(returns)[len(returns) // 2]
        if median_return > best_median_return:
            best_median_return = median_return
            best_allocation = allocation
    return best_allocation, best_median_return

def rebalance_portfolio(allocation, years=30, threshold=0.05):
    """
    Adjusts the allocation based on performance deviation.
    """
    for year in range(1, years + 1):
        simulated_returns = generate_returns(allocation, years=1, simulations=1000)
        median_return = sorted(simulated_returns)[len(simulated_returns) // 2]
        if median_return < threshold:
            allocation, _ = optimize_allocation()
            print(f"Rebalanced in Year {year}: {allocation}")
    return allocation

def estimate_final_corpus():
    """
    Estimates the final corpus based on the best allocation strategy.
    """
    best_allocation, best_return = optimize_allocation()
    final_corpus = sum(generate_returns(best_allocation, simulations=1000)) / 1000
    return best_allocation, final_corpus

if __name__ == "__main__":
    best_strategy, expected_corpus = estimate_final_corpus()
    print(f"Best Asset Allocation: {best_strategy}")
    print(f"Expected Final Corpus: {expected_corpus:.2f}")


In [ ]:
#compute the impact of top-up on final corpus

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Simulation parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
market_dip_threshold = -0.05  # Dip defined as -5% monthly return
top_up_amount = 10000  # Extra ₹10k per dip

# Simulated monthly returns (Equity-heavy NPS)
mean_return = 0.008  # Avg monthly return ~9.6% annualized
volatility = 0.05  # Market volatility

# Running Monte Carlo Simulations
simulations = 10000
results_without_topup = []
results_with_topup = []
total_topup_list = []
topup_return_list = []

for _ in range(simulations):
    corpus_without_topup = 0
    corpus_with_topup = 0
    monthly_contribution_dynamic = monthly_contribution  # Contribution increases yearly
    total_topup = 0  # Track total top-up amount

    for month in range(months):
        # Simulate market return
        market_return = np.random.normal(mean_return, volatility)

        # Without Top-Up: Regular contribution
        corpus_without_topup = corpus_without_topup * (1 + market_return) + monthly_contribution_dynamic

        # With Top-Up: Extra ₹10k investment during dips
        if market_return < market_dip_threshold:
            corpus_with_topup += top_up_amount  # Extra investment
            total_topup += top_up_amount  # Track total amount invested via top-ups

        corpus_with_topup = corpus_with_topup * (1 + market_return) + monthly_contribution_dynamic

        # Increase monthly contribution annually
        if month % 12 == 0:
            monthly_contribution_dynamic *= annual_increase

    results_without_topup.append(corpus_without_topup)
    results_with_topup.append(corpus_with_topup)
    total_topup_list.append(total_topup)
    topup_return_list.append(corpus_with_topup - corpus_without_topup)  # Extra corpus generated from top-ups

# Convert to DataFrame
results_df = pd.DataFrame({
    "Without Top-Up": results_without_topup,
    "With Top-Up": results_with_topup,
    "Total Top-Up Invested": total_topup_list,
    "Top-Up Returns": topup_return_list
})

# Calculate means
mean_without_topup = results_df["Without Top-Up"].mean()
mean_with_topup = results_df["With Top-Up"].mean()
mean_total_topup = results_df["Total Top-Up Invested"].mean()
mean_topup_returns = results_df["Top-Up Returns"].mean()

print(f"Final Corpus Without Top-Up: ₹{mean_without_topup/1e7:.2f} Cr")
print(f"Final Corpus With Top-Up: ₹{mean_with_topup/1e7:.2f} Cr")
print(f"Total Top-Up Invested: ₹{mean_total_topup/1e6:.2f} Lakh")
print(f"Extra Returns from Top-Up: ₹{mean_topup_returns/1e6:.2f} Lakh")

# Visualization
plt.figure(figsize=(12, 6))

# Histogram of Final Corpus (Without & With Top-Up)
plt.subplot(1, 2, 1)
plt.hist(results_df["Without Top-Up"], bins=50, alpha=0.7, color='blue', edgecolor='black', label="Without Top-Up")
plt.hist(results_df["With Top-Up"], bins=50, alpha=0.7, color='green', edgecolor='black', label="With Top-Up")
plt.axvline(mean_without_topup, color='red', linestyle='dashed', linewidth=2, label="Avg Without Top-Up")
plt.axvline(mean_with_topup, color='purple', linestyle='dashed', linewidth=2, label="Avg With Top-Up")
plt.title("Final Corpus Distribution")
plt.xlabel("Corpus (₹)")
plt.ylabel("Frequency")
plt.legend()

# Histogram of Top-Up Returns
plt.subplot(1, 2, 2)
plt.hist(results_df["Top-Up Returns"], bins=50, alpha=0.7, color='orange', edgecolor='black')
plt.axvline(mean_topup_returns, color='red', linestyle='dashed', linewidth=2, label="Avg Extra Returns")
plt.title("Extra Returns Due to Top-Up")
plt.xlabel("Extra Corpus from Top-Up (₹)")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
''' statically check the best allocation strategy and top-up impact
ideally, every time you change allocation say based on dip, top-up should be done.
meaning allocation change & top-up are together not independent of each other.
this we will test in the part cell
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

# Simulation Parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
top_up_base = 10000  # Top-up base amount
dip_levels = [-0.03, -0.05, -0.07, -0.10]  # Dip thresholds for top-up
top_up_multipliers = [1, 2, 3, 5]  # Multiplier for top-ups

# Asset Classes and Parameters
assets = ["Equity", "Bonds", "G-Secs", "Alt Assets"]
returns = {"Equity": 0.008, "Bonds": 0.005, "G-Secs": 0.004, "Alt Assets": 0.010}
volatility = {"Equity": 0.05, "Bonds": 0.02, "G-Secs": 0.01, "Alt Assets": 0.06}

# Simulation Parameters
simulations = 1000
allocation_steps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

# Generate possible asset allocation combinations (that sum to 1)
allocations = [
    list(allocation) for allocation in product(allocation_steps, repeat=4) if sum(allocation) == 1.0
]

best_corpus = 0
best_allocation = None
results_summary = []

# Run simulations for each asset allocation
for allocation in allocations:
    weights = dict(zip(assets, allocation))
    results_without_topup = []
    results_with_topup = []

    for _ in range(simulations):
        corpus_without_topup = 0
        corpus_with_topup = 0
        monthly_contribution_dynamic = monthly_contribution

        for month in range(months):
            # Simulate returns based on asset allocation
            total_return = sum(
                weights[asset] * np.random.normal(returns[asset], volatility[asset]) for asset in assets
            )

            # Without Top-Up
            corpus_without_topup = corpus_without_topup * (1 + total_return) + monthly_contribution_dynamic

            # With Top-Up: Check for dips and apply top-ups
            top_up_amount = 0
            for dip, multiplier in zip(dip_levels, top_up_multipliers):
                if total_return < dip:
                    top_up_amount = top_up_base * multiplier
                else:
                    break

            corpus_with_topup += top_up_amount
            corpus_with_topup = corpus_with_topup * (1 + total_return) + monthly_contribution_dynamic

            # Increase contribution annually
            if month % 12 == 0:
                monthly_contribution_dynamic *= annual_increase

        results_without_topup.append(corpus_without_topup)
        results_with_topup.append(corpus_with_topup)

    # Calculate mean corpus for this allocation
    mean_with_topup = np.mean(results_with_topup)
    results_summary.append((allocation, mean_with_topup))

    # Update best strategy if corpus is higher
    if mean_with_topup > best_corpus:
        best_corpus = mean_with_topup
        best_allocation = allocation

# Display best strategy
print(f"✅ Best Asset Allocation: {dict(zip(assets, best_allocation))}")
print(f"📈 Final Corpus with Optimal Allocation & Top-Up: ₹{best_corpus / 1e7:.2f} Cr")

# Visualizing Top 5 Allocations
top_results = sorted(results_summary, key=lambda x: x[1], reverse=True)[:5]
top_labels = [f"{dict(zip(assets, alloc))}" for alloc, _ in top_results]
top_corpus = [corpus / 1e7 for _, corpus in top_results]

plt.figure(figsize=(10, 6))
plt.barh(top_labels, top_corpus, color='skyblue')
plt.xlabel("Final Corpus (₹ Cr)")
plt.ylabel("Top 5 Asset Allocations")
plt.title("Top 5 Asset Allocations with Top-Up Impact")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
''' combine top up & dynamic allocation - this code didnt work, the logic was conflicting '''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

# Simulation Parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
top_up_base = 10000  # Top-up base amount
dip_levels = [-0.03, -0.05, -0.07, -0.10]  # Dip thresholds
top_up_multipliers = [1, 2, 3, 5]  # Top-up multipliers

# Asset Classes and Parameters
assets = ["Equity", "Bonds", "G-Secs", "Alt Assets"]
returns = {"Equity": 0.008, "Bonds": 0.005, "G-Secs": 0.004, "Alt Assets": 0.010}
volatility = {"Equity": 0.05, "Bonds": 0.02, "G-Secs": 0.01, "Alt Assets": 0.06}

# NPS Allocation Limits (Active Choice)
nps_limits = {
    "Equity": (0, 0.75),
    "Bonds": (0, 1.0),
    "G-Secs": (0, 1.0),
    "Alt Assets": (0, 0.05),
}

# Monte Carlo Parameters
simulations = 1000
allocation_steps = [0.1, 0.2, 0.3, 0.4, 0.5]  # 10%, 20%, 30%, etc.

# Generate Valid Asset Allocations within NPS Limits
valid_allocations = []
for allocation in product(allocation_steps, repeat=4):
    if sum(allocation) == 1.0:
        within_limits = all(
            nps_limits[asset][0] <= allocation[i] <= nps_limits[asset][1]
            for i, asset in enumerate(assets)
        )
        if within_limits:
            valid_allocations.append(allocation)

# Track best strategy
best_corpus = 0
best_allocation = None
results_summary = []

# Run Baseline Simulation (No Top-Up / No Reallocation)
results_baseline = []

for _ in range(simulations):
    corpus_baseline = 0
    monthly_contribution_dynamic = monthly_contribution

    for month in range(months):
        # Fixed Baseline Allocation
        baseline_weights = {"Equity": 0.5, "Bonds": 0.3, "G-Secs": 0.15, "Alt Assets": 0.05}
        baseline_return = sum(
            baseline_weights[asset] * np.random.normal(returns[asset], volatility[asset])
            for asset in assets
        )
        corpus_baseline = corpus_baseline * (1 + baseline_return) + monthly_contribution_dynamic

        # Increase monthly contribution annually
        if month % 12 == 0:
            monthly_contribution_dynamic *= annual_increase

    results_baseline.append(corpus_baseline)

# Average Final Corpus for Baseline
mean_baseline = np.mean(results_baseline)
print(f"🏦 Final Corpus (Baseline without Top-Up/Rebalancing): ₹{mean_baseline / 1e7:.2f} Cr")

# Run Monte Carlo Simulation with Top-Up + Rebalancing
for allocation in valid_allocations:
    weights = dict(zip(assets, allocation))
    results_combined = []

    for _ in range(simulations):
        corpus_combined = 0
        monthly_contribution_dynamic = monthly_contribution
        total_top_up = 0
        top_up_roi = 0

        for month in range(months):
            # Simulate monthly market return based on asset allocation
            total_return = sum(
                weights[asset] * np.random.normal(returns[asset], volatility[asset])
                for asset in assets
            )

            # Top-Up Logic if Market Dips
            top_up_amount = 0
            rebalance_triggered = False

            for dip, multiplier in zip(dip_levels, top_up_multipliers):
                if total_return < dip:
                    top_up_amount = top_up_base * multiplier
                    rebalance_triggered = True
                    break

            corpus_combined += top_up_amount
            corpus_combined = corpus_combined * (1 + total_return) + monthly_contribution_dynamic

            # Track Total Top-Up and its ROI
            if top_up_amount > 0:
                total_top_up += top_up_amount
                top_up_roi += top_up_amount * (1 + total_return)

            # Rebalancing after a Dip if Triggered
            if rebalance_triggered:
                weights = {
                    "Equity": min(weights["Equity"] + 0.1, 0.75),
                    "Bonds": max(weights["Bonds"] - 0.05, 0.1),
                    "G-Secs": max(weights["G-Secs"] - 0.05, 0.1),
                    "Alt Assets": min(weights["Alt Assets"] + 0.05, 0.05),
                }

                # Ensure weights sum to 1 after rebalance
                total_weight = sum(weights.values())
                weights = {k: v / total_weight for k, v in weights.items()}

            # Increase Monthly Contribution Annually
            if month % 12 == 0:
                monthly_contribution_dynamic *= annual_increase

        # Append Corpus + Top-Up ROI
        results_combined.append(corpus_combined + top_up_roi)

    # Average Final Corpus for this Allocation
    mean_combined = np.mean(results_combined)
    results_summary.append((allocation, mean_combined, total_top_up, top_up_roi))

    # Update Best Strategy
    if mean_combined > best_corpus:
        best_corpus = mean_combined
        best_allocation = allocation

# Display Best Strategy
if best_allocation is not None:
    print(f"✅ Best Strategy Allocation: {dict(zip(assets, best_allocation))}")
    print(f"📈 Final Corpus with Optimal Allocation + Top-Up: ₹{best_corpus / 1e7:.2f} Cr")
else:
    print("⚠️ No optimal allocation strategy found. Check allocation limits or input parameters!")

# 📊 Visualization: Comparison Baseline vs Optimal Strategy
labels = ["Baseline (No Top-Up/Rebalancing)", "Optimal Strategy with Top-Up"]
final_corpus_values = [mean_baseline / 1e7, best_corpus / 1e7]

plt.figure(figsize=(8, 6))
plt.bar(labels, final_corpus_values, color=["grey", "green"])
plt.ylabel("Final Corpus (₹ Cr)")
plt.title("Comparison: Baseline vs Optimal Strategy")
plt.show()


In [ ]:
'''new code after i have the working independent code of top-up and allocation strategy
to test the combined impact'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Simulation parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
market_dip_threshold = -0.05  # Dip defined as -5% monthly return
top_up_amount = 10000  # Extra ₹10k per dip
simulations = 10000

# Asset classes and returns
asset_classes = {
    'Equity': (0.12, 0.20),  # Mean return 12%, volatility 20%
    'Corporate Bonds': (0.08, 0.10),
    'Government Bonds': (0.06, 0.05),
    'Alternative Assets': (0.10, 0.15)
}

# Allocation Limits for NPS
min_allocation = {'Equity': 0.2, 'Corporate Bonds': 0.05, 'Government Bonds': 0.05, 'Alternative Assets': 0.0}
max_allocation = {'Equity': 0.75, 'Corporate Bonds': 0.4, 'Government Bonds': 0.4, 'Alternative Assets': 0.2}

# Generate returns with optimal allocation and top-up
def generate_returns(allocation, simulations=10000, apply_topup=True):
    final_values_without_topup = []
    final_values_with_topup = []
    total_topup_list = []
    topup_return_list = []

    for _ in range(simulations):
        corpus_without_topup = 0
        corpus_with_topup = 0
        total_topup = 0
        monthly_contribution_dynamic = monthly_contribution

        for month in range(months):
            # Simulate market return based on asset allocation
            monthly_return = sum(
                allocation[asset] * np.random.normal(asset_classes[asset][0] / 12, asset_classes[asset][1] / np.sqrt(12))
                for asset in allocation
            )

            # Without Top-Up: Regular contribution
            corpus_without_topup = corpus_without_topup * (1 + monthly_return) + monthly_contribution_dynamic

            # With Top-Up: Top-Up triggered during dips
            if apply_topup and monthly_return < market_dip_threshold:
                corpus_with_topup += top_up_amount
                total_topup += top_up_amount

            corpus_with_topup = corpus_with_topup * (1 + monthly_return) + monthly_contribution_dynamic

            # Annual increase in contribution
            if month % 12 == 0:
                monthly_contribution_dynamic *= annual_increase

        # Store results
        final_values_without_topup.append(corpus_without_topup)
        final_values_with_topup.append(corpus_with_topup)
        total_topup_list.append(total_topup)
        topup_return_list.append(corpus_with_topup - corpus_without_topup)

    # Create DataFrame
    results_df = pd.DataFrame({
        "Without Top-Up": final_values_without_topup,
        "With Top-Up": final_values_with_topup,
        "Total Top-Up Invested": total_topup_list,
        "Top-Up Returns": topup_return_list
    })

    return results_df

# Optimize Allocation using Monte Carlo
def optimize_allocation(simulations=100):
    best_allocation = None
    best_corpus = 0

    for _ in range(simulations):
        # Randomly generate allocation within limits
        allocation = {
            asset: random.uniform(min_allocation[asset], max_allocation[asset])
            for asset in asset_classes
        }
        total = sum(allocation.values())
        for key in allocation:
            allocation[key] /= total  # Normalize to ensure allocation sums to 1

        # Run simulation with generated allocation
        results_df = generate_returns(allocation, simulations=1000)
        avg_corpus_with_topup = results_df["With Top-Up"].mean()

        # Track the best allocation that maximizes corpus
        if avg_corpus_with_topup > best_corpus:
            best_corpus = avg_corpus_with_topup
            best_allocation = allocation

    return best_allocation, best_corpus

# Run the combined simulation
best_allocation, best_corpus_with_topup = optimize_allocation()

# Generate final corpus for all scenarios
results_df = generate_returns(best_allocation, simulations=10000)

# Calculate average corpus values
mean_without_topup = results_df["Without Top-Up"].mean()
mean_with_topup = results_df["With Top-Up"].mean()
mean_total_topup = results_df["Total Top-Up Invested"].mean()
mean_topup_returns = results_df["Top-Up Returns"].mean()

print(f"✅ Best Strategy Allocation: {dict(zip(best_allocation.keys(), np.round(list(best_allocation.values()), 2)))}")
print(f"📈 Final Corpus Without Top-Up: ₹{mean_without_topup / 1e7:.2f} Cr")
print(f"💹 Final Corpus With Top-Up: ₹{mean_with_topup / 1e7:.2f} Cr")
print(f"💰 Total Top-Up Invested: ₹{mean_total_topup / 1e6:.2f} Lakh")
print(f"📊 Extra Returns from Top-Up: ₹{mean_topup_returns / 1e6:.2f} Lakh")

# Visualization
plt.figure(figsize=(12, 6))

# Histogram of Final Corpus (Without & With Top-Up)
plt.subplot(1, 2, 1)
plt.hist(results_df["Without Top-Up"], bins=50, alpha=0.7, color='blue', edgecolor='black', label="Without Top-Up")
plt.hist(results_df["With Top-Up"], bins=50, alpha=0.7, color='green', edgecolor='black', label="With Top-Up")
plt.axvline(mean_without_topup, color='red', linestyle='dashed', linewidth=2, label="Avg Without Top-Up")
plt.axvline(mean_with_topup, color='purple', linestyle='dashed', linewidth=2, label="Avg With Top-Up")
plt.title("Final Corpus Distribution")
plt.xlabel("Corpus (₹)")
plt.ylabel("Frequency")
plt.legend()

# Histogram of Top-Up Returns
plt.subplot(1, 2, 2)
plt.hist(results_df["Top-Up Returns"], bins=50, alpha=0.7, color='orange', edgecolor='black')
plt.axvline(mean_topup_returns, color='red', linestyle='dashed', linewidth=2, label="Avg Extra Returns")
plt.title("Extra Returns Due to Top-Up")
plt.xlabel("Extra Corpus from Top-Up (₹)")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
'''now we make it even more dynamic, 
we make the dips to be dynamic instead of a static value'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# 🎯 Simulation Parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
top_up_base = 10000  # Base Top-Up amount during dips
simulations = 10000  # Number of Monte Carlo simulations

# 🎯 Asset Classes & Return Assumptions (Mean Return, Volatility)
asset_classes = {
    'Equity': (0.12, 0.20),  # Mean return 12%, volatility 20%
    'Corporate Bonds': (0.08, 0.10),
    'Government Bonds': (0.06, 0.05),
    'Alternative Assets': (0.10, 0.15)
}

# 🎯 Allocation Limits for NPS Asset Classes
min_allocation = {'Equity': 0.2, 'Corporate Bonds': 0.05, 'Government Bonds': 0.05, 'Alternative Assets': 0.0}
max_allocation = {'Equity': 0.75, 'Corporate Bonds': 0.4, 'Government Bonds': 0.4, 'Alternative Assets': 0.2}

# 🎯 Dynamic Dip Parameters (Dip threshold ranges from 0% to -50%)
dip_min = 0  # No dip (0%)
dip_max = -0.5  # Max dip (-50%)

# 🎯 Dynamic Top-Up Scaling Factors (Higher multipliers for deeper dips)
dip_thresholds = [-0.05, -0.1, -0.2, -0.3, -0.5]
top_up_multipliers = [1, 2, 3, 5, 10]  # Multiplier increases with deeper dips

# 📈 Generate Returns with Dynamic Dip & Top-Up Logic
def generate_dynamic_returns(allocation, simulations=10000):
    """
    Simulates returns for a given asset allocation with dynamic dip detection and top-up.
    """
    final_values_without_topup = []  # Store corpus without top-up
    final_values_with_topup = []  # Store corpus with dynamic top-up
    total_topup_list = []  # Store total top-up amount invested
    topup_return_list = []  # Store additional corpus generated via top-up

    # 🔄 Run multiple simulations
    for _ in range(simulations):
        corpus_without_topup = 0
        corpus_with_topup = 0
        total_topup = 0
        monthly_contribution_dynamic = monthly_contribution  # Contribution grows annually

        # 📅 Loop through each month for the defined number of years
        for month in range(months):
            # 🎢 Simulate market return based on asset allocation
            monthly_return = sum(
                allocation[asset] * np.random.normal(asset_classes[asset][0] / 12, asset_classes[asset][1] / np.sqrt(12))
                for asset in allocation
            )

            # 📉 Set dynamic dip threshold (random dip between 0% and -50%)
            dip_threshold = np.random.uniform(dip_min, dip_max)

            # 🚫 Without Top-Up: Regular contribution
            corpus_without_topup = corpus_without_topup * (1 + monthly_return) + monthly_contribution_dynamic

            # 💡 With Top-Up: Check if dip exceeds threshold → Trigger top-up
            top_up_amount = 0
            for dip, multiplier in zip(dip_thresholds, top_up_multipliers):
                if monthly_return < dip_threshold and monthly_return < dip:
                    top_up_amount = top_up_base * multiplier  # Scale top-up based on dip size
                    break

            # 🧾 Apply top-up and update corpus with top-up
            corpus_with_topup += top_up_amount
            total_topup += top_up_amount
            corpus_with_topup = corpus_with_topup * (1 + monthly_return) + monthly_contribution_dynamic

            # 📈 Increase contribution annually (every 12 months)
            if month % 12 == 0:
                monthly_contribution_dynamic *= annual_increase

        # 📊 Store results after each simulation
        final_values_without_topup.append(corpus_without_topup)
        final_values_with_topup.append(corpus_with_topup)
        total_topup_list.append(total_topup)
        topup_return_list.append(corpus_with_topup - corpus_without_topup)

    # 📝 Create a DataFrame to store results
    results_df = pd.DataFrame({
        "Without Top-Up": final_values_without_topup,
        "With Top-Up": final_values_with_topup,
        "Total Top-Up Invested": total_topup_list,
        "Top-Up Returns": topup_return_list
    })

    return results_df

# 📊 Optimize Asset Allocation to Maximize Returns
def optimize_allocation(simulations=100):
    """
    Optimizes the asset allocation to maximize final corpus.
    """
    best_allocation = None
    best_corpus = 0

    # 🔄 Test different allocations within NPS limits
    for _ in range(simulations):
        # 🎲 Generate random allocation within min-max limits
        allocation = {
            asset: random.uniform(min_allocation[asset], max_allocation[asset])
            for asset in asset_classes
        }

        # 🔢 Normalize allocation to ensure the sum equals 1
        total = sum(allocation.values())
        for key in allocation:
            allocation[key] /= total

        # 📊 Run dynamic dip simulation with current allocation
        results_df = generate_dynamic_returns(allocation, simulations=1000)
        avg_corpus_with_topup = results_df["With Top-Up"].mean()

        # 🔥 Track best allocation based on highest corpus with top-up
        if avg_corpus_with_topup > best_corpus:
            best_corpus = avg_corpus_with_topup
            best_allocation = allocation

    return best_allocation, best_corpus

# 🎯 Run Combined Strategy Simulation
best_allocation, best_corpus_with_topup = optimize_allocation()

# 🧠 Run Simulation with Best Allocation for Final Comparison
results_df = generate_dynamic_returns(best_allocation, simulations=10000)

# 📊 Calculate Final Corpus Stats
mean_without_topup = results_df["Without Top-Up"].mean()
mean_with_topup = results_df["With Top-Up"].mean()
mean_total_topup = results_df["Total Top-Up Invested"].mean()
mean_topup_returns = results_df["Top-Up Returns"].mean()

# 🎉 Print Final Results
print(f"🏄 Best Strategy Allocation: {dict(zip(best_allocation.keys(), np.round(list(best_allocation.values()), 2)))}")
print(f"📈 Final Corpus Without Top-Up: ₹{mean_without_topup / 1e7:.2f} Cr")
print(f"💹 Final Corpus With Top-Up: ₹{mean_with_topup / 1e7:.2f} Cr")
print(f"💰 Total Top-Up Invested: ₹{mean_total_topup / 1e6:.2f} Lakh")
print(f"📊 Extra Returns from Top-Up: ₹{mean_topup_returns / 1e6:.2f} Lakh")

# 📊 Visualization
plt.figure(figsize=(12, 6))

# 📊 Histogram of Corpus With & Without Top-Up
plt.subplot(1, 2, 1)
plt.hist(results_df["Without Top-Up"], bins=50, alpha=0.7, color='blue', edgecolor='black', label="Without Top-Up")
plt.hist(results_df["With Top-Up"], bins=50, alpha=0.7, color='green', edgecolor='black', label="With Top-Up")
plt.axvline(mean_without_topup, color='red', linestyle='dashed', linewidth=2, label="Avg Without Top-Up")
plt.axvline(mean_with_topup, color='purple', linestyle='dashed', linewidth=2, label="Avg With Top-Up")
plt.title("Final Corpus Distribution (With & Without Top-Up)")
plt.xlabel("Corpus (₹)")
plt.ylabel("Frequency")
plt.legend()

# 📊 Histogram of Top-Up Returns
plt.subplot(1, 2, 2)
plt.hist(results_df["Top-Up Returns"], bins=50, alpha=0.7, color='orange', edgecolor='black')
plt.axvline(mean_topup_returns, color='red', linestyle='dashed', linewidth=2, label="Avg Extra Returns")
plt.title("Extra Returns Due to Top-Up (Dynamic Dip)")
plt.xlabel("Extra Corpus from Top-Up (₹)")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
'''even more complex, rebalance every 5 years'''

# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Simulation parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
top_up_base = 10000  # Base top-up amount
dip_min, dip_max = -0.02, -0.10  # Dynamic dip range (-2% to -10%)
mean_return = 0.008  # Average monthly return (~9.6% annualized)
volatility = 0.05  # Market volatility
simulations = 2000  # Number of Monte Carlo simulations

# Asset classes and NPS limits
assets = ['Equity', 'Corporate Bonds', 'Government Bonds', 'Alternative Assets']
limits = {
    'Equity': (0.20, 0.75),
    'Corporate Bonds': (0.05, 0.40),
    'Government Bonds': (0.05, 0.40),
    'Alternative Assets': (0.00, 0.20)
}

# Function to generate random asset allocation within NPS limits
def generate_allocation():
    allocation = {}
    remaining = 1
    for i, asset in enumerate(assets):
        min_limit, max_limit = limits[asset]
        if i == len(assets) - 1:  # Last asset takes remaining allocation
            allocation[asset] = remaining
        else:
            allocation[asset] = np.random.uniform(min_limit, min(max_limit, remaining))
            remaining -= allocation[asset]
    return allocation

# Function to calculate weighted return based on asset allocation
def get_monthly_return(allocation):
    asset_returns = {
        'Equity': np.random.normal(0.01, 0.06),  # Avg 12% annualized with 20% volatility
        'Corporate Bonds': np.random.normal(0.0065, 0.03),  # Avg 8% annualized
        'Government Bonds': np.random.normal(0.0045, 0.02),  # Avg 6% annualized
        'Alternative Assets': np.random.normal(0.0075, 0.04)  # Avg 10% annualized
    }
    return sum(allocation[asset] * asset_returns[asset] for asset in assets)

# Simulation results storage
results_df = pd.DataFrame(columns=["Baseline", "Top-Up", "Optimal Allocation", "Combined"])
total_topup_list = []
topup_return_list = []

# Track allocation and top-up events
rebalance_log = []

# Monte Carlo Simulation
for _ in range(simulations):
    # Initial corpus and contribution
    corpus_baseline = corpus_topup = corpus_optimal = corpus_combined = 0
    monthly_contribution_dynamic = monthly_contribution
    total_topup = 0
    optimal_allocation = generate_allocation()

    for month in range(months):
        year = month // 12 + 1
        market_return_baseline = get_monthly_return({'Equity': 0.50, 'Corporate Bonds': 0.20, 'Government Bonds': 0.20, 'Alternative Assets': 0.10})
        
        # Baseline Corpus (No Top-Up, No Rebalancing)
        corpus_baseline = corpus_baseline * (1 + market_return_baseline) + monthly_contribution_dynamic

        # Top-Up Only Corpus
        market_return_topup = market_return_baseline
        dip_threshold = np.random.uniform(dip_min, dip_max)  # Dynamic dip threshold
        if market_return_topup < dip_threshold:
            corpus_topup += top_up_base  # Add top-up during dip
            total_topup += top_up_base
            rebalance_log.append({
                "Year": year,
                "Month": month % 12 + 1,
                "Event": "Top-Up",
                "Top-Up Amount": top_up_base,
                "Market Return": round(market_return_topup * 100, 2)
            })

        corpus_topup = corpus_topup * (1 + market_return_topup) + monthly_contribution_dynamic

        # Optimal Allocation Corpus (Rebalanced annually)
        market_return_optimal = get_monthly_return(optimal_allocation)
        corpus_optimal = corpus_optimal * (1 + market_return_optimal) + monthly_contribution_dynamic

        # Combined Strategy Corpus (Top-Up + Optimal Allocation + Rebalancing)
        market_return_combined = get_monthly_return(optimal_allocation)
        if market_return_combined < dip_threshold:
            corpus_combined += top_up_base
        corpus_combined = corpus_combined * (1 + market_return_combined) + monthly_contribution_dynamic

        # Annual Contribution Increase & Rebalancing
        if month % 12 == 0:
            monthly_contribution_dynamic *= annual_increase
            if month % 60 == 0:  # Rebalance every 5 years
                old_allocation = optimal_allocation.copy()
                optimal_allocation = generate_allocation()
                rebalance_log.append({
                    "Year": year,
                    "Event": "Rebalance",
                    "Old Allocation": old_allocation,
                    "New Allocation": optimal_allocation
                })

    # Store results in DataFrame
    results_df = results_df.append({
        "Baseline": corpus_baseline,
        "Top-Up": corpus_topup,
        "Optimal Allocation": corpus_optimal,
        "Combined": corpus_combined
    }, ignore_index=True)
    total_topup_list.append(total_topup)
    topup_return_list.append(corpus_topup - corpus_baseline)

# Calculate mean corpus values
mean_baseline = results_df["Baseline"].mean()
mean_topup = results_df["Top-Up"].mean()
mean_optimal = results_df["Optimal Allocation"].mean()
mean_combined = results_df["Combined"].mean()
mean_topup_return = np.mean(topup_return_list)

# Print results
print(f"📈 Baseline Corpus: ₹{mean_baseline / 1e7:.2f} Cr")
print(f"💸 Top-Up Only Corpus: ₹{mean_topup / 1e7:.2f} Cr (Impact: ₹{mean_topup_return / 1e5:.2f} Lakh)")
print(f"📊 Optimal Allocation Corpus: ₹{mean_optimal / 1e7:.2f} Cr")
print(f"🏆 Combined Strategy Corpus: ₹{mean_combined / 1e7:.2f} Cr")

# Print Rebalance and Top-Up Log
print("\n✅ Rebalance and Top-Up Log:")
for log in rebalance_log:
    if log["Event"] == "Top-Up":
        print(f"Year {log['Year']}, Month {log['Month']}: Top-Up of ₹{log['Top-Up Amount']:,} due to market dip of {log['Market Return']}%")
    elif log["Event"] == "Rebalance":
        print(f"Year {log['Year']}: Allocation rebalanced from {log['Old Allocation']} to {log['New Allocation']}")

# Visualization
plt.figure(figsize=(12, 6))

# Histogram of Corpus Distribution
plt.subplot(1, 2, 1)
plt.hist(results_df["Baseline"], bins=50, alpha=0.7, color='blue', edgecolor='black', label="Baseline")
plt.hist(results_df["Top-Up"], bins=50, alpha=0.7, color='green', edgecolor='black', label="Top-Up")
plt.hist(results_df["Optimal Allocation"], bins=50, alpha=0.7, color='orange', edgecolor='black', label="Optimal Allocation")
plt.hist(results_df["Combined"], bins=50, alpha=0.7, color='purple', edgecolor='black', label="Combined")
plt.title("Corpus Distribution Across Strategies")
plt.xlabel("Final Corpus (₹)")
plt.ylabel("Frequency")
plt.legend()

# Histogram of Top-Up Impact
plt.subplot(1, 2, 2)
plt.hist(topup_return_list, bins=50, alpha=0.7, color='orange', edgecolor='black')
plt.axvline(mean_topup_return, color='red', linestyle='dashed', linewidth=2, label="Avg Top-Up Impact")
plt.title("Extra Returns Due to Top-Up")
plt.xlabel("Extra Corpus from Top-Up (₹)")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
'''now rebalnces whenever equity allocation crosses 70%'''
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Simulation parameters
years = 30
months = years * 12
monthly_contribution = 10000  # ₹10k per month
annual_increase = 1.1  # 10% yearly increase
top_up_base = 10000  # Base top-up amount
dip_min, dip_max = -0.10, -0.40  # Dynamic dip range (-10% to -50%)
mean_return = 0.005  # Average monthly return (~9.6% annualized)
volatility = 0.10  # Market volatility
simulations = 1000  # Number of Monte Carlo simulations

# Asset classes and NPS limits
assets = ['Equity', 'Corporate Bonds', 'Government Bonds', 'Alternative Assets']
limits = {
    'Equity': (0.20, 0.75),
    'Corporate Bonds': (0.05, 0.40),
    'Government Bonds': (0.05, 0.40),
    'Alternative Assets': (0.00, 0.20)
}
equity_rebalance_threshold = 0.70  # 70% threshold for rebalancing

# Global cooling period variable (in months)
cooling_period = 6

# Function to generate random asset allocation within NPS limits
def generate_allocation():
    allocation = {}
    remaining = 1
    for i, asset in enumerate(assets):
        min_limit, max_limit = limits[asset]
        if i == len(assets) - 1:  # Last asset takes remaining allocation
            allocation[asset] = remaining
        else:
            allocation[asset] = np.random.uniform(min_limit, min(max_limit, remaining))
            remaining -= allocation[asset]
    return allocation

# Function to calculate weighted return based on asset allocation
def get_monthly_return(allocation):
    asset_returns = {
        'Equity': np.random.normal(0.01, 0.06),  # Avg 12% annualized with 20% volatility
        'Corporate Bonds': np.random.normal(0.0065, 0.03),  # Avg 8% annualized
        'Government Bonds': np.random.normal(0.0045, 0.02),  # Avg 6% annualized
        'Alternative Assets': np.random.normal(0.0075, 0.04)  # Avg 10% annualized
    }
    return sum(allocation[asset] * asset_returns[asset] for asset in assets)

# Function to rebalance allocation when equity > 70%
def rebalance(allocation):
    total = sum(allocation.values())
    new_allocation = {}
    # Reduce equity to 60% during rebalancing
    new_allocation['Equity'] = min(0.60, allocation['Equity'])
    # Distribute remaining across other assets proportionally
    remaining = 1 - new_allocation['Equity']
    for asset in assets:
        if asset != 'Equity':
            new_allocation[asset] = allocation[asset] / (total - allocation['Equity']) * remaining
    return new_allocation

# Simulation results storage
results_df = pd.DataFrame(columns=["Baseline", "Top-Up", "Optimal Allocation", "Combined"])
total_topup_list = []
topup_return_list = []
rebalance_log = []   # To log rebalancing events
topup_log = []       # To log top-up events

# Monte Carlo Simulation
for _ in range(simulations):
    # Reset per simulation variables
    corpus_baseline = corpus_topup = corpus_optimal = corpus_combined = 0
    monthly_contribution_dynamic = monthly_contribution
    total_topup = 0
    last_topup_month = -cooling_period  # so that a top-up can happen in the first month if conditions meet
    topup_growth = 0  # Reset extra growth from top-ups for this simulation
    optimal_allocation = generate_allocation()
    
    for month in range(months):
        # --- Baseline Strategy ---
        # Simulate market return with a fixed allocation for baseline
        market_return_baseline = get_monthly_return({
            'Equity': 0.50, 
            'Corporate Bonds': 0.20, 
            'Government Bonds': 0.20, 
            'Alternative Assets': 0.10
        })
        corpus_baseline = corpus_baseline * (1 + market_return_baseline) + monthly_contribution_dynamic

        # --- Top-Up Only Strategy ---
        market_return_topup = market_return_baseline  # Same return as baseline for comparison
        dip_threshold = np.random.uniform(dip_min, dip_max)  # Dynamic dip threshold
        # Check if conditions for top-up are met
        # Check if conditions for top-up are met
        if market_return_topup < 0 and market_return_topup < dip_threshold:
            if month - last_topup_month >= cooling_period:
                print(f"\n🎯 Month {month + 1}: BEFORE Top-Up, Corpus: ₹{corpus_topup:.2f}")
                corpus_topup += top_up_base  # Add top-up before applying market return
                total_topup += top_up_base
                last_topup_month = month
                print("after top_up", corpus_topup)
                
        
            # Apply 1% immediate boost after top-up
                corpus_topup *= 1.01  # 1% boost after top-up
                print(f"🚀 Month {month + 1}: AFTER Top-Up, Corpus (with boost): ₹{corpus_topup:.2f}")
                # Log the top-up event using the return that triggered it
            topup_log.append({
                    "Month": month + 1,
                    "Top-Up Amount": top_up_base,
                    "Market Return": market_return_topup,
                    "Threshold": dip_threshold,
                    "Reason": f"Market dip of {market_return_topup:.2%} triggered top-up"
                })
        # Now apply the market return and add monthly contribution
        print(f"📊 Month {month + 1}: BEFORE Market Return, Corpus: ₹{corpus_topup:.2f}")
        corpus_topup = corpus_topup * (1 + market_return_topup) + monthly_contribution_dynamic
        print(f"📈 Month {month + 1}: AFTER Market Return, Corpus: ₹{corpus_topup:.2f}")
        # --- Optimal Allocation Strategy ---
        market_return_optimal = get_monthly_return(optimal_allocation)
        # Check if equity exceeds the rebalance threshold; if so, rebalance
        if optimal_allocation['Equity'] > equity_rebalance_threshold:
            new_allocation = rebalance(optimal_allocation)
            rebalance_log.append({
                "Month": month + 1,
                "Old Allocation": optimal_allocation,
                "New Allocation": new_allocation
            })
            optimal_allocation = new_allocation
        corpus_optimal = corpus_optimal * (1 + market_return_optimal) + monthly_contribution_dynamic

        # --- Combined Strategy (Top-Up + Optimal Allocation + Rebalancing) ---
        market_return_combined = get_monthly_return(optimal_allocation)
        if market_return_combined < 0 and market_return_combined < dip_threshold:
            corpus_combined += top_up_base  # Add top-up before compounding
            total_topup += top_up_base
        corpus_combined = corpus_combined * (1 + market_return_combined) + monthly_contribution_dynamic

        # --- Increase Contribution Annually ---
        if (month + 1) % 12 == 0:
            monthly_contribution_dynamic *= annual_increase

        # Optional: Print debug info every 50 months
        #if month % 50 == 0:
         #   print(f"Month {month}: Baseline Corpus: ₹{corpus_baseline:.2f}, Top-Up Corpus: ₹{corpus_topup:.2f}, Difference: ₹{(corpus_topup - corpus_baseline):.2f}")

    # Store the simulation results
    results_df = results_df.append({
        "Baseline": corpus_baseline,
        "Top-Up": corpus_topup,
        "Optimal Allocation": corpus_optimal,
        "Combined": corpus_combined
    }, ignore_index=True)
    total_topup_list.append(total_topup)
    topup_return_list.append(corpus_topup - corpus_baseline)

# Calculate mean corpus values
mean_baseline = results_df["Baseline"].mean()
mean_topup = results_df["Top-Up"].mean()
mean_optimal = results_df["Optimal Allocation"].mean()
mean_combined = results_df["Combined"].mean()
mean_topup_return = np.mean(topup_return_list)

# Print final results
print(f"📈 Baseline Corpus: ₹{mean_baseline / 1e7:.2f} Cr")
print(f"💸 Top-Up Only Corpus: ₹{mean_topup / 1e7:.2f} Cr (Impact: ₹{mean_topup_return / 1e5:.2f} Lakh)")
print(f"📊 Optimal Allocation Corpus: ₹{mean_optimal / 1e7:.2f} Cr")
print(f"🏆 Combined Strategy Corpus: ₹{mean_combined / 1e7:.2f} Cr")

# Print top-up logs (first 5 instances)
print("\n📢 Top-Up Events (First 5 instances):")
for log in topup_log[:5]:
    print(f"Month {log['Month']}: Top-Up of ₹{log['Top-Up Amount']:.2f} due to {log['Reason']}")

# Print total top-up amount and count
total_topup_amount = sum([log["Top-Up Amount"] for log in topup_log])
total_topup_count = len(topup_log)
print(f"\n💡 Total Top-Up Amount: ₹{total_topup_amount / 1e5:.2f} Lakh")
print(f"📅 Total Number of Top-Ups: {total_topup_count} instances")

# Print rebalancing logs (first 5 instances)
print("\n🔄 Rebalancing Events (First 5 instances):")
for log in rebalance_log[:5]:
    print(f"Month {log['Month']}: Rebalanced Allocation from {log['Old Allocation']} to {log['New Allocation']}")

# Visualization
plt.figure(figsize=(12, 6))

# Histogram of Corpus Distribution Across Strategies
plt.subplot(1, 2, 1)
plt.hist(results_df["Baseline"], bins=50, alpha=0.7, color='blue', edgecolor='black', label="Baseline")
plt.hist(results_df["Top-Up"], bins=50, alpha=0.7, color='green', edgecolor='black', label="Top-Up")
plt.hist(results_df["Optimal Allocation"], bins=50, alpha=0.7, color='orange', edgecolor='black', label="Optimal Allocation")
plt.hist(results_df["Combined"], bins=50, alpha=0.7, color='purple', edgecolor='black', label="Combined")
plt.title("Corpus Distribution Across Strategies")
plt.xlabel("Final Corpus (₹)")
plt.ylabel("Frequency")
plt.legend()

# Histogram of Top-Up Impact
plt.subplot(1, 2, 2)
plt.hist(topup_return_list, bins=50, alpha=0.7, color='orange', edgecolor='black')
plt.axvline(mean_topup_return, color='red', linestyle='dashed', linewidth=2, label="Avg Top-Up Impact")
plt.title("Extra Returns Due to Top-Up")
plt.xlabel("Extra Corpus from Top-Up (₹)")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()



In [15]:
'''we missed using the concept of units uptil now, which impact the results of top-up strategy
here we use units'''

'''we missed 10% yearly increase & other asset classes in this, its in the next cell'''

import numpy as np

# Simulation parameters
initial_corpus = 500000  # Starting corpus
monthly_contribution = 10000  # Monthly contribution
years = 30  # Investment period
total_months = years * 12
dip_threshold = -0.10  # Dip trigger for top-up (10% dip)
top_up_base = 10000  # Amount to top-up during dips
cooling_period = 12  # Cooling period before reapplying top-up

# Annual return assumptions
equity_return_avg = 0.12
debt_return_avg = 0.07
volatility_equity = 0.15
volatility_debt = 0.03

# Initial NAV setup
initial_nav_equity = 10
initial_nav_debt = 10

# Optimal allocation ratios
equity_ratio_optimal = 0.75
debt_ratio_optimal = 0.25

# Number of Monte Carlo simulations
num_simulations = 5000

# Lists to store results across simulations
results_baseline, results_topup, results_optimal, results_combined = [], [], [], []

# Lists to store NAV and units for reporting
nav_results, unit_results = [], []

# Monte Carlo simulation loop
np.random.seed(42)  # For reproducibility
for sim in range(num_simulations):
    # Initial corpus and units for each strategy
    corpus_baseline, corpus_topup, corpus_optimal, corpus_combined = (
        initial_corpus,
        initial_corpus,
        initial_corpus,
        initial_corpus,
    )

    # Initial units based on corpus
    units_baseline = initial_corpus / initial_nav_equity
    units_topup = initial_corpus / initial_nav_equity
    units_equity_optimal = (initial_corpus * equity_ratio_optimal) / initial_nav_equity
    units_debt_optimal = (initial_corpus * debt_ratio_optimal) / initial_nav_debt
    units_equity_combined = units_equity_optimal
    units_debt_combined = units_debt_optimal

    # NAVs for equity and debt
    nav_equity_baseline, nav_equity_topup, nav_equity_optimal, nav_equity_combined = (
        initial_nav_equity,
        initial_nav_equity,
        initial_nav_equity,
        initial_nav_equity,
    )
    nav_debt_optimal, nav_debt_combined = initial_nav_debt, initial_nav_debt

    # Top-Up tracking variables
    last_topup_month = -cooling_period
    total_topup = 0

    # Track initial NAV and units for comparison
    initial_navs = {
        "Baseline": nav_equity_baseline,
        "Top-Up": nav_equity_topup,
        "Optimal Allocation": nav_equity_optimal,
        "Combined": nav_equity_combined,
    }
    initial_units = {
        "Baseline": units_baseline,
        "Top-Up": units_topup,
        "Optimal Allocation": units_equity_optimal + units_debt_optimal,
        "Combined": units_equity_combined + units_debt_combined,
    }

    # Run individual simulation
    for month in range(total_months):
        # Monthly growth for equity and debt
        equity_growth = np.random.normal(equity_return_avg / 12, volatility_equity / np.sqrt(12))
        debt_growth = np.random.normal(debt_return_avg / 12, volatility_debt / np.sqrt(12))

        # Update NAVs
        nav_equity_baseline *= (1 + equity_growth)
        nav_equity_topup *= (1 + equity_growth)
        nav_equity_optimal *= (1 + equity_growth)
        nav_equity_combined *= (1 + equity_growth)

        nav_debt_optimal *= (1 + debt_growth)
        nav_debt_combined *= (1 + debt_growth)

        # Monthly contributions
        units_baseline += monthly_contribution / nav_equity_baseline
        units_topup += monthly_contribution / nav_equity_topup
        units_equity_optimal += (monthly_contribution * equity_ratio_optimal) / nav_equity_optimal
        units_debt_optimal += (monthly_contribution * debt_ratio_optimal) / nav_debt_optimal
        units_equity_combined += (monthly_contribution * equity_ratio_optimal) / nav_equity_combined
        units_debt_combined += (monthly_contribution * debt_ratio_optimal) / nav_debt_combined

        # Top-Up Logic
        market_return_topup = (nav_equity_topup / initial_nav_equity) - 1
        if market_return_topup < 0 and market_return_topup < dip_threshold:
            if month - last_topup_month >= cooling_period:
                topup_units_added = top_up_base / nav_equity_topup
                units_topup += topup_units_added
                total_topup += top_up_base
                last_topup_month = month

    # Final corpus values
    corpus_baseline = units_baseline * nav_equity_baseline
    corpus_topup = units_topup * nav_equity_topup
    corpus_optimal = (units_equity_optimal * nav_equity_optimal) + (units_debt_optimal * nav_debt_optimal)
    corpus_combined = (units_equity_combined * nav_equity_combined) + (units_debt_combined * nav_debt_combined)

    # Store results of this simulation
    results_baseline.append(corpus_baseline)
    results_topup.append(corpus_topup)
    results_optimal.append(corpus_optimal)
    results_combined.append(corpus_combined)

    # Track final NAVs and units
    final_navs = {
        "Baseline": nav_equity_baseline,
        "Top-Up": nav_equity_topup,
        "Optimal Allocation": nav_equity_optimal,
        "Combined": nav_equity_combined,
    }
    final_units = {
        "Baseline": units_baseline,
        "Top-Up": units_topup,
        "Optimal Allocation": units_equity_optimal + units_debt_optimal,
        "Combined": units_equity_combined + units_debt_combined,
    }

    # Store NAV and unit results for each run
    nav_results.append(final_navs)
    unit_results.append(final_units)

# Summary of results
print(f"✅ Simulated {num_simulations} runs successfully!")
print(f"📈 Average Corpus Results after 30 years:")
print(f"📈 Baseline: ₹{np.mean(results_baseline) / 1e7:.2f} Cr ± ₹{np.std(results_baseline) / 1e7:.2f} Cr")
print(f"💸 Top-Up: ₹{np.mean(results_topup) / 1e7:.2f} Cr ± ₹{np.std(results_topup) / 1e7:.2f} Cr")
print(f"📊 Optimal Allocation: ₹{np.mean(results_optimal) / 1e7:.2f} Cr ± ₹{np.std(results_optimal) / 1e7:.2f} Cr")
print(f"🏆 Combined Strategy: ₹{np.mean(results_combined) / 1e7:.2f} Cr ± ₹{np.std(results_combined) / 1e7:.2f} Cr")

# Display NAV and units for the first simulation
print("\n🔎 NAV and Units Comparison for First Run:")
for strategy in ["Baseline", "Top-Up", "Optimal Allocation", "Combined"]:
    print(f"📈 {strategy}:")
    print(f"   - Initial NAV: ₹{initial_navs[strategy]:.4f}, Final NAV: ₹{nav_results[0][strategy]:.4f}")
    print(f"   - Initial Units: {initial_units[strategy]:.4f}, Final Units: {unit_results[0][strategy]:.4f}")

# Display percentile ranges
print("\n📊 Percentile Ranges (25th, 50th, 75th):")
print(f"📈 Baseline: {np.percentile(results_baseline, [25, 50, 75])}")
print(f"💸 Top-Up: {np.percentile(results_topup, [25, 50, 75])}")
print(f"📊 Optimal Allocation: {np.percentile(results_optimal, [25, 50, 75])}")
print(f"🏆 Combined Strategy: {np.percentile(results_combined, [25, 50, 75])}")



✅ Simulated 5000 runs successfully!
📈 Average Corpus Results after 30 years:
📈 Baseline: ₹5.20 Cr ± ₹4.16 Cr
💸 Top-Up: ₹5.22 Cr ± ₹4.16 Cr
📊 Optimal Allocation: ₹4.31 Cr ± ₹3.12 Cr
🏆 Combined Strategy: ₹4.31 Cr ± ₹3.12 Cr

🔎 NAV and Units Comparison for First Run:
📈 Baseline:
   - Initial NAV: ₹10.0000, Final NAV: ₹150.3830
   - Initial Units: 50000.0000, Final Units: 182509.4458
📈 Top-Up:
   - Initial NAV: ₹10.0000, Final NAV: ₹150.3830
   - Initial Units: 50000.0000, Final Units: 182509.4458
📈 Optimal Allocation:
   - Initial NAV: ₹10.0000, Final NAV: ₹150.3830
   - Initial Units: 50000.0000, Final Units: 186738.7678
📈 Combined:
   - Initial NAV: ₹10.0000, Final NAV: ₹150.3830
   - Initial Units: 50000.0000, Final Units: 186738.7678

📊 Percentile Ranges (25th, 50th, 75th):
📈 Baseline: [25843857.94863245 40419085.20898984 64746614.59327623]
💸 Top-Up: [25919960.23664065 40556160.62248126 64884348.60748158]
📊 Optimal Allocation: [23490848.52809788 34409513.00668599 52662954.52357881]
🏆 

In [16]:
'''the baseline code'''

import numpy as np
import pandas as pd

# --- Constants ---
years = 30
months = years * 12
initial_corpus = 1000000  # Starting corpus
monthly_contribution = 10000  # Initial monthly contribution
top_up_base = 10000  # Base amount for top-ups
top_up_scale_factor = 0.1  # Increase by 10% per 1% dip beyond 10%
cooling_period = 12  # Minimum months between top-ups
dip_threshold = -0.10  # Dip threshold to trigger top-up
max_dip_to_top_up = -0.30  # Stop top-up beyond 30% dip
annual_growth_rate = 0.10  # Monthly contribution growth per year

# --- Asset Class Properties ---
assets = {
    "Equity": {"mean": 0.01, "volatility": 0.057, "min": 0.20, "max": 0.75},
    "Corporate Bonds": {"mean": 0.0065, "volatility": 0.015, "min": 0.05, "max": 0.40},
    "Government Bonds": {"mean": 0.0045, "volatility": 0.01, "min": 0.05, "max": 0.40},
    "Alternative Assets": {"mean": 0.0075, "volatility": 0.02, "min": 0.0, "max": 0.20},
}

# --- Initial Allocation ---
initial_allocation = {"Equity": 0.60, "Corporate Bonds": 0.15, "Government Bonds": 0.15, "Alternative Assets": 0.10}

# --- Variables ---
nav = {asset: 10.0 for asset in assets}
units = {asset: (initial_corpus * initial_allocation[asset]) / nav[asset] for asset in assets}
top_up_units = {asset: 0 for asset in assets}
last_topup_month = -cooling_period
total_topup = 0

# --- Simulation ---
np.random.seed(42)
monthly_returns = {asset: np.random.normal(assets[asset]["mean"], assets[asset]["volatility"], months) for asset in assets}

# --- Simulation Loop ---
for month in range(months):
    # Grow monthly contribution by 10% annually
    if month % 12 == 0 and month != 0:
        monthly_contribution *= (1 + annual_growth_rate)
        
    # Apply returns and contributions
    for asset in assets:
        nav[asset] *= (1 + monthly_returns[asset][month])
        units[asset] += (monthly_contribution * initial_allocation[asset]) / nav[asset]

    # Check for Top-Up Conditions
    equity_dip = monthly_returns["Equity"][month]
    if equity_dip < dip_threshold and equity_dip >= max_dip_to_top_up and (month - last_topup_month) >= cooling_period:
        dip_magnitude = abs(equity_dip * 100) - 10  # Beyond 10%
        scale_factor = 1 + (dip_magnitude * top_up_scale_factor)
        scaled_top_up = top_up_base * scale_factor
        for asset in assets:
            top_up_units[asset] += (scaled_top_up * initial_allocation[asset]) / nav[asset]
        total_topup += scaled_top_up
        last_topup_month = month

# --- Final Corpus Calculations ---
final_corpus = {asset: (units[asset] + top_up_units[asset]) * nav[asset] for asset in assets}
total_corpus = sum(final_corpus.values())

# --- Percentile Analysis ---
num_simulations = 5000
corpus_results = []

for _ in range(num_simulations):
    simulated_nav = {asset: 10.0 for asset in assets}
    simulated_units = {asset: units[asset] + top_up_units[asset] for asset in assets}

    for month in range(months):
        for asset in assets:
            simulated_nav[asset] *= (1 + np.random.normal(assets[asset]["mean"], assets[asset]["volatility"]))
    
    corpus_results.append(sum(simulated_units[asset] * simulated_nav[asset] for asset in assets))

percentiles = np.percentile(corpus_results, [10, 25, 50, 75, 90])
final_results = {
    "final_corpus": final_corpus,
    "total_corpus": total_corpus,
    "nav": nav,
    "percentiles": percentiles
}

final_results


{'final_corpus': {'Equity': 70004378.820165,
  'Corporate Bonds': 7623218.963726994,
  'Government Bonds': 7163078.330166298,
  'Alternative Assets': 8868545.34190408},
 'total_corpus': 93659221.45596236,
 'nav': {'Equity': 222.0962142455335,
  'Corporate Bonds': 87.88634160836942,
  'Government Bonds': 73.19933967822207,
  'Alternative Assets': 226.7986739153913},
 'percentiles': array([3.56139185e+07, 5.01961827e+07, 8.24877389e+07, 1.46547470e+08,
        2.63504622e+08])}

In [20]:
'''output display improvement in the baseline code'''

import numpy as np

# Simulation settings
num_simulations = 5000
initial_corpus = 1000000
corpus_growth_rates = {"Equity": 0.12, "Corporate Bonds": 0.08, "Government Bonds": 0.06, "Alternative Assets": 0.10}
volatility_rates = {"Equity": 0.2, "Corporate Bonds": 0.05, "Government Bonds": 0.03, "Alternative Assets": 0.15}
allocation_optimal = {"Equity": 0.60, "Corporate Bonds": 0.15, "Government Bonds": 0.15, "Alternative Assets": 0.10}

# Results placeholders
results_baseline, results_topup, results_optimal, results_combined = [], [], [], []
nav_results, unit_results = [], []

# Simulation loop
for _ in range(num_simulations):
    corpus = {"Baseline": initial_corpus, "Top-Up": initial_corpus, "Optimal Allocation": initial_corpus, "Combined": initial_corpus}
    navs = {asset: 10 for asset in allocation_optimal.keys()}
    units = {asset: initial_corpus / 10 for asset in allocation_optimal.keys()}
    monthly_contribution = 5000

    for month in range(360):  # 30 years * 12 months
        # Annual contribution growth by 10%
        if month % 12 == 0 and month > 0:
            monthly_contribution *= 1.10

        # Simulate market fluctuations and growth per asset class
        market_change = np.random.normal(0, 0.02)

        for asset in allocation_optimal.keys():
            growth = np.random.normal(corpus_growth_rates[asset] / 12, volatility_rates[asset] / np.sqrt(12))
            navs[asset] *= (1 + growth + market_change)

        # Monthly contributions added to corpus
        for strategy in ["Baseline", "Top-Up", "Optimal Allocation", "Combined"]:
            corpus[strategy] += monthly_contribution
            for asset in allocation_optimal.keys():
                units[asset] += (monthly_contribution * allocation_optimal[asset]) / navs[asset]

        # Dynamic top-up logic with 30% dip safeguard
        if market_change < -0.1 and market_change > -0.3:
            top_up_amount = 10000 * (1 + abs(market_change * 10))
            corpus["Top-Up"] += top_up_amount
            for asset in allocation_optimal.keys():
                units[asset] += (top_up_amount * allocation_optimal[asset]) / navs[asset]

        # Update corpus values based on latest NAV & units
        for strategy in ["Baseline", "Top-Up", "Optimal Allocation", "Combined"]:
            corpus[strategy] = sum(units[asset] * navs[asset] for asset in allocation_optimal.keys())

    # Store results
    results_baseline.append(corpus["Baseline"])
    results_topup.append(corpus["Top-Up"])
    results_optimal.append(corpus["Optimal Allocation"])
    results_combined.append(corpus["Combined"])

    # Save NAV and unit comparison for first run
    nav_results.append({asset: navs[asset] for asset in allocation_optimal.keys()})
    unit_results.append({asset: units[asset] for asset in allocation_optimal.keys()})

# Store final NAV and units per strategy
final_navs, final_units = {}, {}
for strategy in ["Baseline", "Top-Up", "Optimal Allocation", "Combined"]:
    final_navs[strategy] = {asset: navs[asset] for asset in allocation_optimal.keys()}
    final_units[strategy] = {asset: units[asset] for asset in allocation_optimal.keys()}

# Summary of results
print(f"✅ Simulated {num_simulations} runs successfully!")
print(f"📈 Average Corpus Results after 30 years:")
print(f"📈 Baseline: ₹{np.mean(results_baseline) / 1e7:.2f} Cr ± ₹{np.std(results_baseline) / 1e7:.2f} Cr")
print(f"💸 Top-Up: ₹{np.mean(results_topup) / 1e7:.2f} Cr ± ₹{np.std(results_topup) / 1e7:.2f} Cr")
print(f"📊 Optimal Allocation: ₹{np.mean(results_optimal) / 1e7:.2f} Cr ± ₹{np.std(results_optimal) / 1e7:.2f} Cr")
print(f"🏆 Combined Strategy: ₹{np.mean(results_combined) / 1e7:.2f} Cr ± ₹{np.std(results_combined) / 1e7:.2f} Cr")

# Display NAV and units for the first simulation
print("\n🔎 NAV and Units Comparison for First Run:")
for strategy in ["Baseline", "Top-Up", "Optimal Allocation", "Combined"]:
    print(f"📈 {strategy}:")
    for asset in allocation_optimal.keys():
        print(f"   - {asset} NAV: ₹{final_navs[strategy][asset]:.4f}, Units: {final_units[strategy][asset]:.4f}")

# Display percentile ranges
print("\n📊 Percentile Ranges (25th, 50th, 75th):")
print(f"📈 Baseline: {np.percentile(results_baseline, [25, 50, 75])}")
print(f"💸 Top-Up: {np.percentile(results_topup, [25, 50, 75])}")
print(f"📊 Optimal Allocation: {np.percentile(results_optimal, [25, 50, 75])}")
print(f"🏆 Combined Strategy: {np.percentile(results_combined, [25, 50, 75])}")


✅ Simulated 5000 runs successfully!
📈 Average Corpus Results after 30 years:
📈 Baseline: ₹21.53 Cr ± ₹14.67 Cr
💸 Top-Up: ₹21.53 Cr ± ₹14.67 Cr
📊 Optimal Allocation: ₹21.53 Cr ± ₹14.67 Cr
🏆 Combined Strategy: ₹21.53 Cr ± ₹14.67 Cr

🔎 NAV and Units Comparison for First Run:
📈 Baseline:
   - Equity NAV: ₹247.0306, Units: 515909.8614
   - Corporate Bonds NAV: ₹48.3818, Units: 336766.7316
   - Government Bonds NAV: ₹41.5683, Units: 350674.4834
   - Alternative Assets NAV: ₹75.4629, Units: 195517.1009
📈 Top-Up:
   - Equity NAV: ₹247.0306, Units: 515909.8614
   - Corporate Bonds NAV: ₹48.3818, Units: 336766.7316
   - Government Bonds NAV: ₹41.5683, Units: 350674.4834
   - Alternative Assets NAV: ₹75.4629, Units: 195517.1009
📈 Optimal Allocation:
   - Equity NAV: ₹247.0306, Units: 515909.8614
   - Corporate Bonds NAV: ₹48.3818, Units: 336766.7316
   - Government Bonds NAV: ₹41.5683, Units: 350674.4834
   - Alternative Assets NAV: ₹75.4629, Units: 195517.1009
📈 Combined:
   - Equity NAV: ₹247.0

In [22]:
'''in the above code, the units & corpus is same for all, trying to see if its fixed in this ver'''

import numpy as np
import copy

# Simulation settings
num_simulations = 5000
initial_corpus = 1000000
corpus_growth_rates = {
    "Equity": 0.12,
    "Corporate Bonds": 0.08,
    "Government Bonds": 0.06,
    "Alternative Assets": 0.10
}
volatility_rates = {
    "Equity": 0.2,
    "Corporate Bonds": 0.05,
    "Government Bonds": 0.03,
    "Alternative Assets": 0.15
}

allocation_optimal = {
    "Equity": 0.60,
    "Corporate Bonds": 0.15,
    "Government Bonds": 0.15,
    "Alternative Assets": 0.10
}
equal_allocation = {
    asset: 0.25 for asset in corpus_growth_rates
}

# Results placeholders
results_baseline, results_topup, results_optimal, results_combined = [], [], [], []
nav_results, unit_results = [], []
final_navs = {}
final_units = {}

# Simulation loop
for _ in range(num_simulations):
    # Initialize NAVs and units per asset
    base_navs = {asset: 10.0 for asset in corpus_growth_rates}
    base_units = {asset: initial_corpus * equal_allocation[asset] / base_navs[asset] for asset in corpus_growth_rates}
    opt_units = {asset: initial_corpus * allocation_optimal[asset] / base_navs[asset] for asset in corpus_growth_rates}

    # Deep copies per strategy
    nav_baseline = copy.deepcopy(base_navs)
    nav_topup = copy.deepcopy(base_navs)
    nav_optimal = copy.deepcopy(base_navs)
    nav_combined = copy.deepcopy(base_navs)

    units_baseline = copy.deepcopy(base_units)
    units_topup = copy.deepcopy(base_units)
    units_optimal = copy.deepcopy(opt_units)
    units_combined = copy.deepcopy(opt_units)

    monthly_contribution = 5000

    for month in range(360):
        # Annual increment in monthly contribution
        if month % 6 == 0 and month > 0:
            monthly_contribution *= 1.10

        # Market fluctuation
        market_change = np.random.normal(0, 0.02)

        # Update NAVs for each strategy
        for asset in corpus_growth_rates:
            for nav_dict in [nav_baseline, nav_topup, nav_optimal, nav_combined]:
                growth = np.random.normal(
                    corpus_growth_rates[asset] / 12,
                    volatility_rates[asset] / np.sqrt(12)
                )
                nav_dict[asset] *= (1 + growth + market_change)

        # Contribution and top-up logic
        for asset in corpus_growth_rates:
            # Baseline
            units_baseline[asset] += (monthly_contribution * equal_allocation[asset]) / nav_baseline[asset]
            # Top-Up
            units_topup[asset] += (monthly_contribution * equal_allocation[asset]) / nav_topup[asset]
            # Optimal
            units_optimal[asset] += (monthly_contribution * allocation_optimal[asset]) / nav_optimal[asset]
            # Combined
            units_combined[asset] += (monthly_contribution * allocation_optimal[asset]) / nav_combined[asset]

        # Top-up only for Top-Up and Combined strategy (market dip trigger + 30% safeguard)
        if -0.3 < market_change < -0.1:
            topup_amount = 10000 * (1 + abs(market_change * 10))
            for asset in corpus_growth_rates:
                # Top-Up
                units_topup[asset] += (topup_amount * equal_allocation[asset]) / nav_topup[asset]
                # Combined
                units_combined[asset] += (topup_amount * allocation_optimal[asset]) / nav_combined[asset]

    # Final corpus per strategy
    corpus_baseline = sum(units_baseline[asset] * nav_baseline[asset] for asset in corpus_growth_rates)
    corpus_topup = sum(units_topup[asset] * nav_topup[asset] for asset in corpus_growth_rates)
    corpus_optimal = sum(units_optimal[asset] * nav_optimal[asset] for asset in corpus_growth_rates)
    corpus_combined = sum(units_combined[asset] * nav_combined[asset] for asset in corpus_growth_rates)

    results_baseline.append(corpus_baseline)
    results_topup.append(corpus_topup)
    results_optimal.append(corpus_optimal)
    results_combined.append(corpus_combined)

    # Save NAV and unit comparison for first run
    if len(nav_results) == 0:
        final_navs["Baseline"] = nav_baseline
        final_navs["Top-Up"] = nav_topup
        final_navs["Optimal Allocation"] = nav_optimal
        final_navs["Combined"] = nav_combined

        final_units["Baseline"] = units_baseline
        final_units["Top-Up"] = units_topup
        final_units["Optimal Allocation"] = units_optimal
        final_units["Combined"] = units_combined

# Summary of results
print(f"✅ Simulated {num_simulations} runs successfully!")
print(f"📈 Average Corpus Results after 30 years:")
print(f"📈 Baseline: ₹{np.mean(results_baseline) / 1e7:.2f} Cr ± ₹{np.std(results_baseline) / 1e7:.2f} Cr")
print(f"💸 Top-Up: ₹{np.mean(results_topup) / 1e7:.2f} Cr ± ₹{np.std(results_topup) / 1e7:.2f} Cr")
print(f"📊 Optimal Allocation: ₹{np.mean(results_optimal) / 1e7:.2f} Cr ± ₹{np.std(results_optimal) / 1e7:.2f} Cr")
print(f"🏆 Combined Strategy: ₹{np.mean(results_combined) / 1e7:.2f} Cr ± ₹{np.std(results_combined) / 1e7:.2f} Cr")

# Display NAV and units for the first simulation
print("\n🔎 NAV and Units Comparison for First Run:")
for strategy in ["Baseline", "Top-Up", "Optimal Allocation", "Combined"]:
    print(f"📈 {strategy}:")
    for asset in corpus_growth_rates:
        print(f"   - {asset} NAV: ₹{final_navs[strategy][asset]:.4f}, Units: {final_units[strategy][asset]:.4f}")

# Display percentile ranges
print("\n📊 Percentile Ranges (25th, 50th, 75th):")
print(f"📈 Baseline: {np.percentile(results_baseline, [25, 50, 75])}")
print(f"💸 Top-Up: {np.percentile(results_topup, [25, 50, 75])}")
print(f"📊 Optimal Allocation: {np.percentile(results_optimal, [25, 50, 75])}")
print(f"🏆 Combined Strategy: {np.percentile(results_combined, [25, 50, 75])}")


✅ Simulated 5000 runs successfully!
📈 Average Corpus Results after 30 years:
📈 Baseline: ₹18.58 Cr ± ₹5.26 Cr
💸 Top-Up: ₹18.62 Cr ± ₹5.34 Cr
📊 Optimal Allocation: ₹21.43 Cr ± ₹10.38 Cr
🏆 Combined Strategy: ₹21.32 Cr ± ₹10.08 Cr

🔎 NAV and Units Comparison for First Run:
📈 Baseline:
   - Equity NAV: ₹97.2025, Units: 354053.0713
   - Corporate Bonds NAV: ₹45.5603, Units: 809626.1872
   - Government Bonds NAV: ₹22.9819, Units: 1211069.2955
   - Alternative Assets NAV: ₹93.8110, Units: 592219.0761
📈 Top-Up:
   - Equity NAV: ₹73.0816, Units: 617635.1128
   - Corporate Bonds NAV: ₹43.4091, Units: 729690.0686
   - Government Bonds NAV: ₹20.6581, Units: 1396009.9780
   - Alternative Assets NAV: ₹108.2088, Units: 556447.7700
📈 Optimal Allocation:
   - Equity NAV: ₹50.1220, Units: 1362482.8759
   - Corporate Bonds NAV: ₹34.7346, Units: 517278.7127
   - Government Bonds NAV: ₹23.0981, Units: 740961.4693
   - Alternative Assets NAV: ₹269.9156, Units: 95438.7150
📈 Combined:
   - Equity NAV: ₹53.171